# Handling big data
## Lecture objectives

1. Introduce some of the strategies to handle large data sets
2. Show how to profile the performance of particular functions

Big data is often characterized by the 3 "Vs": velocity, volume, and variety.

We've talked implicitly about the **variety** throughout the course — how to clean, join, and otherwise wrangle datasets. Here, we'll focus on issues of **volume**. In short, what can you do when a dataset doesn't fit into memory, or your code takes too long to run.

Here, I'll set out a hierarchy of approaches to dealing with large datasets. Start with the simplest, and move on to the next group of solutions as needed.

* Testing and scaling up
* Running overnight
* Finding a secondary computer
* Profiling
* Being economical with data types
* Sampling

These strategies are all about being efficient and thoughtful with your analysis. They don't need any special procedures or coding, and some of the strategies are self explanatory.

*Testing and scaling up*: get your code working on a tiny subset of the data, before running it on your full dataset. This seems obvious, but will save you a lot of time. For example: run your code on Ventura County data before doing Los Angeles County, or take a small city in the county.

*Running overnight*: Again, this seems obvious. But remember: your time is much more valuable than the computer's. You could spend the time to make your code more efficient, or you could just leave your computer running in the kitchen or in a closet while you sleep / go to the beach / do something fun.

*Finding a secondary computer*: Maybe you have an old laptop with a cracked screen, an erratic keyboard, and a battery that holds about 15 minutes of charge? This is the perfect machine to do some web scraping or similar tasks in the background. Leave it running for a few days/weeks/months and plug in an external hard drive. 

The next few strategies deserve a bit more explanation. We'll take each in turn.

### Profiling

Profiling means identifying the slowest parts of your code, and thinking about how to speed it up. Sometimes, that will be obvious. But in other cases, you need to profile your code. 

For example, imagine you have a long text document and you want to identify stopwords. You might want to loop over each row of the `DataFrame` using `iterrows()`. Let's try this using a word list kindly hosted by Prof. Eric Price at MIT. [Here are the details.](https://stackoverflow.com/questions/18834636/random-word-generator-python)

In [ ]:
import pandas as pd
import requests

word_site = "https://www.mit.edu/~ecprice/wordlist.10000"
response = requests.get(word_site)
wordDf = pd.DataFrame(response.content.splitlines(), columns=['word'])

print(len(wordDf))
wordDf.head()

Let's add a column to indicate whether each word is a stopword.

Then, we can loop over each row of the dataframe, and apply a function to that row. Our function will set our `is_stopword` column to `True` if the word is a stopword. (This is a really inefficient way of doing things!)

Then, we can use the `%timeit` magic function to see how long it takes to run that line. (A magic function is preceded by `%` - it basically helps you run or analyze your code.)

In [ ]:
from nltk.corpus import stopwords

# initialize the column
wordDf['is_stopword'] = None

# write the function
def exclude_stopwords(wdf):
    for idx, row in wdf.iterrows():
        if row['word'] in stopwords.words('english'):
            wdf.loc[idx, 'is_stopword'] = True
        else:
            wdf.loc[idx, 'is_stopword'] = False
    return wdf

           
# to use %timeit, just put it in front of any function
%timeit newdf = exclude_stopwords(wordDf)

That took more than 1 second per loop on my computer...not a big deal, but it might matter if you are dealing with tens of thousands of long documents.

What about using `apply` with a `lambda` function? For me, it takes just over half the time.

In [ ]:
%timeit wordDf['is_stopword'] = wordDf.word.apply(lambda x: x in stopwords.words('english'))

What else could we do to speed this up? Perhaps accessing `stopword.words()` each time incurs some overhead. What if we access this once and store it in a separate variable?

Here, we use `%%timeit` to time the whole cell rather than a single line.

In [ ]:
%%timeit
swords = stopwords.words('english')
wordDf['is_stopword'] = wordDf.word.apply(lambda x: x in swords)

Much faster!

### Being economical with data types

Data types refer to how an object (integer, string, etc.) is stored by Python. `pandas` generally uses the underlying `numpy` data types, [which are described here](https://numpy.org/doc/stable/user/basics.types.html).

Why should you care about data types? Often, it doesn't matter - if you have integers, storing them as a float will usually work just as well. But we've seen several instances where we need to convert the data type.

In particular, numbers can be stored as strings, but we need to convert them to integers or floats to do arithmetic.

And census geoidentifiers (e.g. tract) can be stored as numbers, but that loses the leading zero, so they are hard to join if one table has the geoid as a string.

In [ ]:
x = '1'
y = '2'
print(x*y)  # fails because they are strings

In [ ]:
print(int(x)*int(y))   # converting to integer works

The other major time when you have to worry about data types is to increase the efficiency of your data storage. In general, the order from most- to least-efficient is:
* boolean (True or False)
* integer
* float
* string / object

The relevant consideration is how many *bits* they consume. For example, an integer can be represented in various ways:
* `int64` is the default, which can represent integers from  from -9223372036854775808 to +9223372036854775807 
* `int32` consumes half as much space, but can only represent -2147483648 to +2147483647
* `int16` and `int8` are also possibilities, if you are sure that you won't have large numbers
* You can also have unsigned integers (positive numbers only)

Similar logic applies to floats: `float64`, `float32`, etc. Here, it's the precision rather than the range that's affected.

We can use `info()` to get the memory consumption of a dataframe. Let's compare the different data types.

In [ ]:
import pandas as pd
import numpy as np

# create a dataframe with one column of ones
df = pd.DataFrame(np.ones(50000), columns=['ones'])

df.info()

In [ ]:
df['ones'] = df.ones.astype('int8')
df.info()

We reduce the size of our dataframe from 391k to 49k – an eightfold reduction (which makes sense as we are going from 64 bit to 8 bit).

Here's a less trivial example, using the voting dataset that we used before.

In [ ]:
df = pd.read_csv('data/c037_g20_sov_data_by_g20_srprec.csv')
df.info()

Let's look at the maximum number of votes per precinct, to see what data type might be appropriate.

Here, we use `.max()` to get the maximum value within each column (after dropping the non-numeric `srprec` column), and then `.max()` again to get the maximum across columns.

In [ ]:
df.drop(columns=['srprec']).max()

In [ ]:
df.drop(columns=['srprec']).max().max()

That won't fit into `int16` (-32767 to +32767), but we could use `int32`.

In [ ]:
for col in df.columns:
    if df[col].dtype != 'object': # skip the string column (srprec)
        df[col] = df[col].astype('int32')
df.info()

So we halve the memory usage. Let's check to see whether our data is unaffected.

In [ ]:
df.drop(columns=['srprec']).max().max()

That wouldn't be the case if we converted to (say) `int16`. Beware!

In [ ]:
print(df['SENDEM01'].max())
print(df['SENDEM01'].astype('int16').max())

Note that `pandas` can skip columns, skip rows, and force particular data types when reading in a `csv` file. Check out the options.

In [ ]:
pd.read_csv?

For example, we could read only the Proposition 14 (yes/no) and precinct columns, and read the numeric columns as `int32`.

The `usecols` argument gives us the subset of columns. The `dtype` argument is a dictionary of data types.

Note that we don't specify a type for `PR_14_Y`, so it gets read in as `int64`.

In [ ]:
df = pd.read_csv('data/c037_g20_sov_data_by_g20_srprec.csv', 
                 usecols=['srprec', 'PR_14_N', 'PR_14_Y'],
                 dtype={'srprec':'str', 'PR_14_N':'int32'})
df.info()

### Sampling
Sampling is often useful because there are diminishing returns (in terms of predictive power) to the size of your dataset. In statistical terms, remember that your standard errors go down by $1/\sqrt{N}$, not by $1/N$.

So if you have a million observations, you *might* get similar results through using a sample of half that size. This is likely to be the case for statistical models (e.g. logistic regression). It might not hold for machine learning models which are more flexible in the way that they consider interactions.

Let's recreate the ADU prediction model that we saw in the classification notebook a couple of weeks ago.

In [ ]:
# all this code is identical to what we had before

# read in and join parcels and permits
permits = pd.read_csv('data/ADU_permits.csv') 
parcels = pd.read_csv('data/parcels.csv')
permits.dropna(subset=['Assessor Book', 'Assessor Page','Assessor Parcel'], inplace=True)
permits = permits[permits['Assessor Parcel']!='***']
permits['APN'] = (permits['Assessor Book'].astype(int).astype(str).str.zfill(4) + '-' 
                   + permits['Assessor Page'].astype(int).astype(str).str.zfill(3) + '-'
                   + permits['Assessor Parcel'].astype(int).astype(str).str.zfill(3))
permits = permits.groupby('APN').first()
parcels = parcels.groupby('APN').first()
joinedDf = parcels.join(permits, how='left')

# add dependent variable and dummies
joinedDf['hasADU'] = joinedDf['# of Accessory Dwelling Units'].apply(lambda x: 0 if pd.isnull(x) else 1)
dummies1 = pd.get_dummies(joinedDf.UseType, prefix='usetype_')  # creates a dataframe of dummies
dummies2 = pd.get_dummies(joinedDf.UseDescription, prefix='usedesc_')
joinedDf = joinedDf.join(dummies1).join(dummies2) 

# split the data and fit the model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay

xvars = dummies1.columns.tolist() + dummies2.columns.tolist() + ['YearBuilt1', 
         'Units1', 'Bedrooms1', 'Bathrooms1', 'SQFTmain1', 'Roll_LandValue', 
         'Roll_ImpValue', 'Roll_LandBaseYear', 'Roll_ImpBaseYear', 'CENTER_LAT', 'CENTER_LON']
yvar = 'hasADU'
df_to_fit = joinedDf[xvars+[yvar]].dropna()
X_train, X_test, y_train, y_test = train_test_split(df_to_fit[xvars], df_to_fit[yvar], test_size = 0.25, random_state = 1)

rf = RandomForestClassifier(n_estimators = 10, random_state = 1)  # 10 estimators to save time
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

What if we just estimate this model on half the data? We can use the `sample` function in `pandas`.

In [ ]:
df_to_fit = df_to_fit.sample(frac=0.5) # take 50% sample
    
# this code is identical
X_train, X_test, y_train, y_test = train_test_split(df_to_fit[xvars], df_to_fit[yvar], test_size = 0.25, random_state = 1)

rf = RandomForestClassifier(n_estimators = 10, random_state = 1)  # 10 estimators to save time
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

This isn't so bad. (Remember all of the entries in the confusion matrix should have half the observations, because we have half the sample size.)

However, a better way forward here might be to:
1. Estimate the model on a sample of the data
2. Look at the feature importances
3. Run the model on the full data, but excluding some of the variables that don't increase predictive performance (i.e., low importance)

I'll leave that for you as an exercise.

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Start simple! Most "big" datasets can be handled on any laptop through judicious use of datatypes, profiling your code, and working on a chunk of your data at one time.</li>
    <li>You will rarely need to do more than this, but the UCLA Hoffman2 cluster and cloud service providers are there if you need them.</li>
</ul>
</div>